# Quickstart: An example from the article
The example from the paper is built-in into a **semopy** package as well as data for it. The model's description and the respective data can be retrieved via invoking *get_model* and *get_data* methods from the *example* submodule of the package.

In [1]:
import numpy as np
import semopy
from semopy.example import get_data, get_model

np.random.seed(2019)
model_desc = get_model()
data = get_data()

In [2]:
print(model_desc)

# structural part
eta3 ~ x1 + x2
eta4 ~ x3
x3 ~ eta1 + eta2 + x1 + x4
x4 ~ eta4
x5 ~ x4
# measurement part
eta1 =~ y1 + y2 + y3
eta2 =~ y3
eta3 =~ y4 + y5
eta4 =~ y4 + y6
# additional covariances
eta2 ~~   x2
y5 ~~   y6


Now, we can create an instant of *Model* from *model_desc* and load data into it.

In [3]:
from semopy import Model

model = Model(model_desc)
model.load_dataset(data)

Now we can create an *Optimizer* by passing it *model*. In fact, we can create as many instances of *Optimizer* as we want. Each of them can perform an independent optimization sequence.
Let's say that we want to see estimates provided by minimising Wishart Maximumul Likelihood Ratio, ULS and GLS:

In [4]:
from semopy import Optimizer

opt_mlw = Optimizer(model)
opt_uls = Optimizer(model)
opt_gls = Optimizer(model)

# And now, we run the optimisation sequences.
lf_mlw = opt_mlw.optimize(objective='MLW') # Although MLW is default, we still provide it here for clarity.
lf_uls = opt_uls.optimize(objective='ULS')
lf_gls = opt_gls.optimize(objective='GLS')

print('Resultant objective functions'' values are:')
print('MLW: {:.3f}, ULS: {:.3f}, GLS: {:.3f}'.format(lf_mlw, lf_uls, lf_gls))

Resultant objective functions values are:
MLW: 0.059, ULS: 38.077, GLS: 1.274


Let's also try minimisng the MLW objective but instead of using default SLSQP nonlinear solver we will try using Adam with chunk_size=100 and num_epochs=2000:

In [5]:
opt_mlw_adam = Optimizer(model)
lf_mlw_adam = opt_mlw_adam.optimize(objective='MLW', method='Adam', chunk_size=100, num_epochs=2000)
print('MLW after Adam: {:.3f}'.format(lf_mlw_adam))

MLW after Adam: 0.062


Take a notice that one can't compare results for the same model based on the value of different loss functions. Fit indices (we will compute then in the end of this notebook) are a valid measure.

Also, it can be seen from the code above that we can in fact run another optimisation sequences for the same *Optimizer*, using previous parameters' estimates as starting values:

In [6]:
lf_mlw_adam_slsqp = opt_mlw_adam.optimize(method='SLSQP')
print('MLW after Adam after SLSQP: {:.3f}'.format(lf_mlw_adam_slsqp))

MLW after Adam after SLSQP: 0.058


The *inspector* module of **semopy** contains *inspect* method that is used to retrieve information on parameters' estimates in a user-friendly manner. It has two modes of display - 'list' (the default one) and 'mx'. Let's try the 'list' first:

In [7]:
from semopy.inspector import inspect

print(inspect(opt_mlw, mode='list'))

    lval  op  rval         Value          SE       Z-score       P-value
9     y2  =~  eta1 -2.291988e+00    0.137061 -1.672240e+01  0.000000e+00
10    y3  =~  eta1  2.899419e+00    6.722738  4.312854e-01  6.662608e-01
11    y5  =~  eta3  2.142304e+00    0.082960  2.582341e+01  0.000000e+00
12    y6  =~  eta4 -1.571480e+00    0.018857 -8.333662e+01  0.000000e+00
0   eta3   ~    x1  1.900383e+00    0.121796  1.560303e+01  0.000000e+00
1   eta3   ~    x2  1.755860e+00    0.117675  1.492128e+01  0.000000e+00
2   eta4   ~    x3 -2.256180e+00    0.051714 -4.362820e+01  0.000000e+00
3     x3   ~  eta1 -9.608590e+00   14.768075 -6.506325e-01  5.152837e-01
4     x3   ~  eta2 -6.381047e+00   59.402283 -1.074209e-01  9.144551e-01
6     x3   ~    x1 -4.638153e-01    0.149184 -3.109024e+00  1.877066e-03
5     x3   ~    x4  5.336651e-02    0.051518  1.035887e+00  3.002548e-01
7     x4   ~  eta4  1.278433e+00    0.015698  8.143877e+01  0.000000e+00
8     x5   ~    x4 -7.959493e-01    0.003783 -2.103

We might also want to take a peek at starting values:

In [8]:
print(inspect(opt_mlw, mode='list', what='start'))

    lval  op  rval       Value         SE     Z-score       P-value
9     y2  =~  eta1   -1.374311   5.533779   -0.248349  8.038640e-01
10    y3  =~  eta1    1.171560   0.011298  103.699329  0.000000e+00
11    y5  =~  eta3    0.317345   2.974069    0.106704  9.150238e-01
12    y6  =~  eta4   -1.297488   0.819493   -1.583280  1.133576e-01
0   eta3   ~    x1    0.000000   0.297482    0.000000  1.000000e+00
1   eta3   ~    x2    0.000000   0.290641    0.000000  1.000000e+00
2   eta4   ~    x3    0.000000   0.087064    0.000000  1.000000e+00
3     x3   ~  eta1    0.000000   3.104311    0.000000  1.000000e+00
4     x3   ~  eta2    0.000000   8.551759    0.000000  1.000000e+00
6     x3   ~    x1    0.000000   0.121851    0.000000  1.000000e+00
5     x3   ~    x4    0.000000   0.014745    0.000000  1.000000e+00
7     x4   ~  eta4    0.000000  65.358666    0.000000  1.000000e+00
8     x5   ~    x4    0.000000   0.035804    0.000000  1.000000e+00
13  eta1  ~~  eta1    0.050000   0.221167    0.2

The other mode of display is 'mx'. That's it, matrices with parameters values mapped to their positions will be printed:

In [9]:
print(inspect(opt_mlw, mode='mx'))

Beta:
         eta1      eta2  eta3      eta4       x3        x4   x5        x1  \
eta1  0.00000  0.000000   0.0  0.000000  0.00000  0.000000  0.0  0.000000   
eta2  0.00000  0.000000   0.0  0.000000  0.00000  0.000000  0.0  0.000000   
eta3  0.00000  0.000000   0.0  0.000000  0.00000  0.000000  0.0  1.900383   
eta4  0.00000  0.000000   0.0  0.000000 -2.25618  0.000000  0.0  0.000000   
x3   -9.60859 -6.381047   0.0  0.000000  0.00000  0.053367  0.0 -0.463815   
x4    0.00000  0.000000   0.0  1.278433  0.00000  0.000000  0.0  0.000000   
x5    0.00000  0.000000   0.0  0.000000  0.00000 -0.795949  0.0  0.000000   
x1    0.00000  0.000000   0.0  0.000000  0.00000  0.000000  0.0  0.000000   
x2    0.00000  0.000000   0.0  0.000000  0.00000  0.000000  0.0  0.000000   

           x2  
eta1  0.00000  
eta2  0.00000  
eta3  1.75586  
eta4  0.00000  
x3    0.00000  
x4    0.00000  
x5    0.00000  
x1    0.00000  
x2    0.00000  
Lambda:
        eta1  eta2      eta3     eta4   x3   x4   x5   

The *stats* module has various methods to calculate statistics and fit indices. However, there is a method *gather_statistics* that invokes them all:

In [10]:
from semopy.stats import gather_statistics

s = gather_statistics(opt_mlw)
print(s)

SEMStatistics(dof=36.0, ml=-380.39716346685407, fit_val=0.05907399655746026, chi2=(29.53699827873013, 0.7681624209713601), dof_baseline=57.0, chi2_baseline=7715.373580628611, rmsea=0, cfi=1.0008439130911055, gfi=0.9961716697227869, agfi=0.9505507339193311, nfi=0.996171669722787, tli=1.001336195727584, aic=820.7943269337081, bic=947.2325698863739, params=[ParametersStatistics(value=1.9003834083994189, se=0.12179576081425694, zscore=15.603034093260225, pvalue=0.0), ParametersStatistics(value=1.7558604439788938, se=0.11767496005333505, zscore=14.921275037468183, pvalue=0.0), ParametersStatistics(value=-2.256180218182484, se=0.051713805521650585, zscore=-43.62819938358449, pvalue=0.0), ParametersStatistics(value=-9.608589536238718, se=14.768074538688987, zscore=-0.6506325188883902, pvalue=0.5152837333812579), ParametersStatistics(value=-6.381047300354492, se=59.40228335954013, zscore=-0.10742090942417759, pvalue=0.9144550671613287), ParametersStatistics(value=0.053366514991665656, se=0.051

A particular fit index/statistic can be invoked from *stats* module directly avoiding excess computations. For instance, let's say we want to calculate a GFI:

In [12]:
from semopy.stats import calc_gfi

print(calc_gfi(opt_gls))
print('MLW: {:.3f}, ULS: {:.3f}, GLS: {:.3f}, MLW after Adam after SLSQP: {:.3f}'.format(calc_gfi(opt_mlw),
                                                                                         calc_gfi(opt_uls),
                                                                                         calc_gfi(opt_gls),
                                                                                         calc_gfi(opt_mlw_adam)))

0.7026737337457353
MLW: 0.996, ULS: 1.000, GLS: 0.703, MLW after Adam after SLSQP: 0.996
